### Library

In [43]:
import re
import glob
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Dataset

In [44]:
file_paths = glob.glob('D:/ta_project/data/raw/*.csv')
if not file_paths:
    raise FileNotFoundError("No CSV files found in the specified directory.")

data = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)
data.to_csv('D:/ta_project/data/processed/combined_data.csv', index=False)
print(f"✅ Successfully combined {len(file_paths)} CSV files")

✅ Successfully combined 8 CSV files


In [45]:
df = pd.read_csv('D:ta_project/data/processed/combined_data.csv')
df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'D:ta_project/data/processed/combined_data.csv'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739 entries, 0 to 738
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   No                       739 non-null    int64 
 1   Model Pakaian            739 non-null    object
 2   Jenis Bahan              739 non-null    object
 3   Waktu Pengerjaan (hari)  739 non-null    int64 
 4   Detail Tambahan          739 non-null    object
 5   Harga Jasa (Rp)          739 non-null    object
dtypes: int64(2), object(4)
memory usage: 34.8+ KB


In [ ]:
print(df.isnull().sum())

No                         0
Model Pakaian              0
Jenis Bahan                0
Waktu Pengerjaan (hari)    0
Detail Tambahan            0
Harga Jasa (Rp)            0
dtype: int64


### Data Cleaning

In [ ]:
if 'No' in df.columns:
    df.drop(labels=['No'], axis=1, inplace=True)

In [ ]:
df = df.rename(columns={
    'Model Pakaian': 'model',
    'Jenis Bahan': 'bahan',
    'Waktu Pengerjaan (hari)': 'waktu_pengerjaan',
    'Detail Tambahan': 'ornamen',
    'Harga Jasa (Rp)': 'harga_jasa'
})

In [ ]:
def clean_price(price):
    cleaned = re.sub(r'[^\d]', '', str(price)) # regex used
    return int(cleaned) if cleaned else None

df['harga_jasa'] = df['harga_jasa'].apply(clean_price)
df = df.dropna(subset=['harga_jasa'])

In [ ]:
model_mapping = {
    'Gaun midi': 'midi dress',
    'Gaun maxi': 'maxi dress'
}

df['model'] = df['model'].replace(model_mapping)

In [ ]:
cat_cols = ['model', 'bahan', 'ornamen']

for col in cat_cols:
    df[col] = df[col].str.lower().str.strip()

In [ ]:
# Ornament keyword mapping
ornament_keywords = {
    'bordir': ['bordir', 'bordit'],
    'payet': ['payet', 'payat'],
    'renda': ['renda'],
    'opneisel': ['opneisel', '0pneisel'],
    'none': ['-']
}

# Detect ornament labels
def labels_detection(text):
    if pd.isna(text):
        return []
    labels = set()
    for label, keywords in ornament_keywords.items():
        if any(kw in text for kw in keywords):
            labels.add(label)
    return list(labels)

# Apply detection and store as list
df['ornament_list'] = df['ornamen'].apply(labels_detection)

In [ ]:
# Filter rows with only one ornament
df = df[df['ornament_list'].apply(len) == 1].copy()

# Convert list back to string
df['ornamen'] = df['ornament_list'].apply(lambda x: x[0] if x else 'none')
df.drop(columns=['ornament_list'], inplace=True)

In [ ]:
print("✅ Preprocessing complete! Final dataset shape:", df.shape)

✅ Preprocessing complete! Final dataset shape: (678, 5)


### Train Test Split

In [ ]:
# Split data
X = df.drop('harga_jasa', axis=1)
y = df['harga_jasa']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

### Modelling

In [ ]:
# Linear Regression
trans = ColumnTransformer(
    transformers=[
        ('col', OneHotEncoder(sparse_output=False, drop='first'), ['model', 'bahan', 'ornamen'])
    ], 
    remainder='passthrough'
)

lr = LinearRegression()

pipe = Pipeline([
    ('transform', trans),
    ('model', lr)
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

In [ ]:
print('R2 Score: ', r2_score(y_test, y_pred))
print('MAE: ', mean_absolute_error(y_test, y_pred))

R2 Score:  0.6643660303808256
MAE:  54182.824033706194


In [ ]:
# K-Neighbors Regressor
knr = KNeighborsRegressor(n_neighbors=3)

pipe = Pipeline([
    ('trans', trans),
    ('knr', knr)
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

In [ ]:
print('R2 Score: ', r2_score(y_test, y_pred))
print('MAE: ', mean_absolute_error(y_test, y_pred))

R2 Score:  0.5406427613680985
MAE:  63229.16666666668


In [ ]:
# Random Forest Regressor
rfr = RandomForestRegressor(n_estimators=100,
                              random_state=3,
                              max_samples=0.5,
                              max_features=0.75,
                              max_depth=15)

pipe1 = Pipeline([
    ('transform', trans),
    ('model', rfr)
])

pipe1.fit(X_train, y_train)

y_pred = pipe1.predict(X_test)

In [ ]:
print('R2 Score: ', r2_score(y_test, y_pred))
print('MAE: ', mean_absolute_error(y_test, y_pred))

R2 Score:  0.6738052251181561
MAE:  49969.292717086835


In [ ]:
# Gradient Boosting Regressor
gbr = GradientBoostingRegressor(n_estimators=500)

pipe = Pipeline([
    ('trans', trans),
    ('gbr', gbr)
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

In [ ]:
print('R2 Score: ', r2_score(y_test, y_pred))
print('MAE: ', mean_absolute_error(y_test, y_pred))

R2 Score:  0.6157751313813198
MAE:  53090.629567175805


In [ ]:
def predict_price(model_input, bahan_input, ornamen_input, waktu_input):
    df_input = pd.DataFrame([{
        'model': model_input,
        'bahan': bahan_input,
        'ornamen': ornamen_input,
        'waktu_pengerjaan': waktu_input
    }])
    
    # Prediksi langsung pakai pipeline
    pred = pipe1.predict(df_input)[0]
    rentang = (int(pred - 100000), int(pred + 100000))
    return f"Estimasi harga jasa jahit: Rp{rentang[0]:,} - Rp{rentang[1]:,}"

In [ ]:
# Contoh prediksi
print(predict_price("kebaya tradisional", "brokat", "payet", 7))

Estimasi harga jasa jahit: Rp320,458 - Rp520,458


In [ ]:
# Save model
joblib.dump(pipe1, 'D:/ta_project/models/model.pkl')

['C:/Users/Kayaki/ta_project/models/model.pkl']